In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from lxml import etree
from lxml.html import fromstring, HTMLParser
import numpy as np

In [3]:
# return df
def mercari(id_or_name, shop_name, in_type):
    if (in_type == 'name'):
        url_part = re.sub(' ','%20', id_or_name)
        keys = re.split(' ', id_or_name)
    else:
        url_part = re.sub('/','%20', id_or_name)
        keys = re.findall('\d+', id_or_name)
    url = 'https://buyee.jp/'+shop_name + '/search?keyword=' + url_part +'&status=all'
    print (url)
    p = []
    n = []
    l = []
    s = []
    st = []
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    a = soup.find("ul", class_="item-lists")
    if a:
        a = a.find_all("li", class_="list")
        for i in a:
            name = i.find("img")["alt"]
            if (re.search(keys[0], name) and re.search(keys[1],name)):
                sold = i.find("div" ,class_= "soldOut__text")
                if sold:
                    s.append("sold")
                else:
                    s.append("available")
                p.append(i.find("p", class_="price-fx").text)
                n.append(name)
                l.append(i.find("a")["href"])
                st.append(shop_name)
    df = pd.DataFrame({'price':p,'sold':s,'name':n,'shop':st,'link':l})
    return df

In [4]:
# return s u (list)
def m_price(name):
    url_part = re.sub(' ','%20', name)
    keys = re.split(' ', name)
    url = 'https://buyee.jp/'+'mercari' + '/search?keyword=' + url_part +'&status=all'
    s = []
    u = []
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    a = soup.find("ul", class_="item-lists")
    count = 0
    limit = len(a)
    if (limit>6):
        limit = limit/2

    if a:
        a = a.find_all("li", class_="list")
        for i in a:
            price = 0
            if (re.search(keys[0], name) and re.search(keys[1],name)):
                sold = i.find("div" ,class_= "soldOut__text")
                price = (i.find("p", class_="price-fx").text)
                price = re.sub("[^0-9.]*",'', price)
                if sold:
                    s.append(price)
                else:
                    u.append(price)
                count+=1
            if (count >= limit-1):
                return s, u


In [5]:
#return price, yu4, yu5, name
def yuyutei(input):
    re.sub('/','%2F', input)
    yu_url = 'https://yuyu-tei.jp/game_ws/sell/sell_price.php?name='+input+'&rare=&type&kizu=0'
    s = requests.Session()
    headers = {'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7'}
    s.headers.update(headers)
    source = s.get(yu_url)
    soup = BeautifulSoup(source.content, 'lxml')
    card_list = soup.find("ul", class_="card_list")
    name = card_list.find("img")["alt"]
    price = card_list.find("p", class_='price').text
    price = re.findall("[0-9]*", price)[1]
    name = re.findall("\S[^(]*", name) 
    yuyu_name = re.sub('["“”#]',' ',name[0])
    return price, int(price)*0.04, int(price)*0.05, name[0]

In [6]:
df = pd.read_excel('raw.xlsx')
id = df['id']
final = pd.DataFrame(columns=['name','yen','yu4','yu5','1_sold','2_sold','avg_sold','1_unsold','2_unsold','avg_unsold'])
for i in id:
    price, yu4, yu5, name = yuyutei(i)
    print(name)
    s1=s2=u1=u2=avg_s=avg_u = 0
    if (name != 'NowPrinting'):
        s, u = m_price(name)
        s = np.array(s).astype(float)
        u = np.array(u).astype(float)
        s_len = len(s)
        u_len = len(u)
        if (s_len >=1):
            s_1 = s[0]
            avg_s = s.mean()
        if (s_len >=2):
            s_2 = s[1]
        if (u_len >=1):
            u_1 = u[0]
            avg_u = u.mean()
        if (u_len >=2):
            u_2 = u[1]    
    row = pd.DataFrame([[name,price,yu4,yu5,s_1,s_2,avg_s,u_1,u_2,avg_u]],columns=final.columns)
    final = final.append(row)


SSP 今夜は一緒に
SSP アインクラッド キリト&アスナ
SP 兄妹の絆 リーファ
SEC 命を懸けたデスゲーム
SP 幸福の花冠 ユイ
SP 恥ずかしい姿 愛音
SP 愛おしの光 レム
SP 異世界常夏生活 レム
SP キミと願うミライ 沙花叉クロヱ
NowPrinting
SP キミと願うミライ 常闇トワ
SP キミと願うミライ 一伊那尓栖
SP キミと願うミライ 戌神ころね
SP #あくあーと 湊あくあ
SP #ぺこらーと 兎田ぺこら
SP 奇跡的相性 かぐや
SP “愛の告白”奥沢美咲
SP “MIRAI TICKET”桜内 梨子
SP “ネコミミモード”ネメシス


In [10]:
final = final.set_index(id)
final

,name,yen,yu4,yu5,1_sold,2_sold,avg_sold,1_unsold,2_unsold,avg_unsold
id,,,,,,,,,,
SAO/S71-028SSP,SSP 今夜は一緒に,158000,6320.0,7900.0,6344.40,8431.53,8147.315312,9675.53,8984.42,98725.202857
SAO/S71-003SSP,SSP アインクラッド キリト&アスナ,39800,1592.0,1990.0,1105.73,1963.10,2655.573333,2073.32,3109.94,4519.830000
SAO/S100-025SP,SP 兄妹の絆 リーファ,34800,1392.0,1740.0,1658.63,1658.63,875.917419,1520.42,1520.42,2787.448889
SAO/S100-020SEC,SEC 命を懸けたデスゲーム,49800,1992.0,2490.0,2487.94,2418.84,2929.266296,2487.94,5528.84,4463.112500
SAO/S100-075SP,SP 幸福の花冠 ユイ,4980,199.2,249.0,345.53,207.32,391.326765,324.81,345.53,283.573333
Shh/W62-073SP,SP 恥ずかしい姿 愛音,49800,1992.0,2490.0,2487.94,2487.94,3905.443500,3421.00,7464.00,4768.666667
RZ/S46-T43SP,SP 愛おしの光 レム,198000,7920.0,9900.0,7740.42,6531.00,10358.635200,27644.42,8260.16,14585.782000
RZ/S55-060SP,SP 異世界常夏生活 レム,29800,1192.0,1490.0,1181.80,621.94,1223.804074,3434.81,1313.11,2307.611667
HOL/WE36-31SP,SP キミと願うミライ 沙花叉クロヱ,39800,1592.0,1990.0,2031.83,1866.00,1198.120000,217.70,691.11,1883.134000


In [8]:
final.to_excel('final.xlsx')

In [9]:
    url = 'https://www.ebay.com/sch/' + re.sub('/','%2F',input) 
    key = re.findall('\d+', input )
    p = []
    n = []
    l = []
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    a = soup.find_all("div", class_="s-item__info clearfix", limit = 11)
    for i in a:
    name = i.find("div", class_="s-item__title").text
    if re.search(key[0],name) and re.search(key[1],name):
        price = i.find("span", class_='s-item__price').text
        link = i.find("a", href = True)['href']
        p.append(price)
        n.append(name)
        l.append(link)
    df = pd.DataFrame({ 'price':p, 'name':n,'link':l})     
    df.to_csv('data.csv') 
    df      

IndentationError: expected an indented block (1034756446.py, line 10)